## NMS AYARLARINI HALLEDELİM


In [1]:
from pathlib import Path
import os, sys
from datetime import datetime

sys.path = [p for p in sys.path if p and p != r"C:\HysoExpect"]

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))
os.chdir(PROJECT_ROOT)

best_pt = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\outputs\runs\finetune_y8s_832_lr1e-3_nomosaic\weights\best.pt")
dataset_root = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced")
test_images = dataset_root / "images" / "test"
test_labels = dataset_root / "labels" / "test"

base_out = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection")

from src.predict import predict_folder
from src.eval_count import eval_count


def run_conf_nms(conf: float, tag: str, nms_type: str, diou_thr: float = 0.60, nms_iou: float = 0.50):
    out_root = base_out / f"outputs_{tag}"
    pred_out = out_root / "predictions_txt"
    rep_out  = out_root / "reports"
    stats_csv = out_root / "nms_stats.csv"

    out_root.mkdir(parents=True, exist_ok=True)
    pred_out.mkdir(parents=True, exist_ok=True)
    rep_out.mkdir(parents=True, exist_ok=True)

    stats, per_image = predict_folder(
        weights=str(best_pt),
        images_dir=str(test_images),
        out_dir=str(pred_out),
        conf=conf,
        iou=0.6,
        imgsz=832,
        device="cuda:0",
        nms_type=nms_type,
        diou_thr=diou_thr,
        nms_iou=nms_iou,
        class_agnostic=True,
        max_det=300,
        stats_path=str(stats_csv),
    )

    txt_count = len(list(pred_out.glob("*.txt")))

    df, summary = eval_count(
        weights=str(best_pt),
        images_dir=str(test_images),
        labels_dir=str(test_labels),
        report_dir=str(rep_out),
        conf=conf,
        iou=0.6,
        imgsz=832,
        device="cuda:0",
        pred_dir=str(pred_out),
    )

    log_path = out_root / "run.log"
    with open(log_path, "a", encoding="utf-8") as f:
        f.write("\n" + "="*80 + "\n")
        f.write(f"time: {datetime.now().isoformat()}\n")
        f.write(f"conf: {conf}\n")
        f.write(f"nms_type: {nms_type}\n")
        f.write(f"diou_thr: {diou_thr}\n")
        f.write(f"nms_iou: {nms_iou}\n")
        f.write(f"txt_written: {txt_count}\n")
        f.write("nms_stats:\n")
        for k, v in stats.items():
            f.write(f"{k}: {v}\n")
        f.write("summary:\n")
        f.write(str(summary) + "\n")

    print(f"\n=== DONE: outputs_{tag} ===")
    print(f"predictions_txt: {pred_out} (txt={txt_count})")
    print("NMS stats:", stats)
    print(summary)

    return stats, summary

In [2]:
from datetime import datetime

conf = 0.39
stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
for thr in [0.40, 0.50, 0.60, 0.70]:
    tag = f"conf39_diou{int(round(thr*100)):02d}_{stamp}"
    run_conf_nms(conf, tag, nms_type="diou", diou_thr=thr, nms_iou=0.50)


=== DONE: outputs_conf39_diou40_20260113_031504 ===
predictions_txt: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\outputs_conf39_diou40_20260113_031504\predictions_txt (txt=700)
NMS stats: {'n_images': 700, 'raw_total': 21598, 'kept_total': 18592, 'removed_total': 3006, 'avg_raw_per_image': 30.854285714285716, 'avg_kept_per_image': 26.56, 'avg_removed_per_image': 4.2942857142857145, 'nms_type': 'diou', 'conf': 0.39, 'diou_thr': 0.4, 'nms_iou': 0.5, 'imgsz': 832, 'device': 'cuda:0'}
              metric         value
0                MAE      1.667143
1               RMSE      4.365612
2  MAPE(ignore gt=0)      0.070015
3           N_images    700.000000
4           GT_total  18605.000000
5         Pred_total  18592.000000

=== DONE: outputs_conf39_diou50_20260113_031504 ===
predictions_txt: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\outputs_conf39_diou50_20260113_031504\predictions_txt (txt=700)
NMS stats: {'n_images': 700, 'raw_total': 21598, 'kept_total': 18602, 'removed_tot

In [3]:
from pathlib import Path
p = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\outputs_conf39_diou60\predictions_txt")
print("exists:", p.exists())
print("txt:", len(list(p.glob("*.txt"))))

exists: False
txt: 0


In [5]:
run_conf_nms(0.39, "final_conf39_diou50", nms_type="diou", diou_thr=0.50, nms_iou=0.50)


=== DONE: outputs_final_conf39_diou50 ===
predictions_txt: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\outputs_final_conf39_diou50\predictions_txt (txt=700)
NMS stats: {'n_images': 700, 'raw_total': 21598, 'kept_total': 18602, 'removed_total': 2996, 'avg_raw_per_image': 30.854285714285716, 'avg_kept_per_image': 26.574285714285715, 'avg_removed_per_image': 4.28, 'nms_type': 'diou', 'conf': 0.39, 'diou_thr': 0.5, 'nms_iou': 0.5, 'imgsz': 832, 'device': 'cuda:0'}
              metric         value
0                MAE      1.667143
1               RMSE      4.367575
2  MAPE(ignore gt=0)      0.070168
3           N_images    700.000000
4           GT_total  18605.000000
5         Pred_total  18602.000000


({'n_images': 700,
  'raw_total': 21598,
  'kept_total': 18602,
  'removed_total': 2996,
  'avg_raw_per_image': 30.854285714285716,
  'avg_kept_per_image': 26.574285714285715,
  'avg_removed_per_image': 4.28,
  'nms_type': 'diou',
  'conf': 0.39,
  'diou_thr': 0.5,
  'nms_iou': 0.5,
  'imgsz': 832,
  'device': 'cuda:0'},
               metric         value
 0                MAE      1.667143
 1               RMSE      4.367575
 2  MAPE(ignore gt=0)      0.070168
 3           N_images    700.000000
 4           GT_total  18605.000000
 5         Pred_total  18602.000000)

## Tam inference a gerek yok tamamen bilgi edinmeye çalışıyoruz


In [6]:
from datetime import datetime

stamp = datetime.now().strftime("%Y%m%d_%H%M%S")

conf_list = [0.385, 0.390, 0.395]
thr_list  = [0.48, 0.49, 0.50, 0.51, 0.52, 0.53, 0.54, 0.55]

for conf in conf_list:
    for thr in thr_list:
        tag = f"grid_c{int(conf*1000)}_diou{int(thr*100)}_{stamp}"
        run_conf_nms(conf, tag, nms_type="diou", diou_thr=thr, nms_iou=0.50)


=== DONE: outputs_grid_c385_diou48_20260113_033217 ===
predictions_txt: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\outputs_grid_c385_diou48_20260113_033217\predictions_txt (txt=700)
NMS stats: {'n_images': 700, 'raw_total': 21741, 'kept_total': 18630, 'removed_total': 3111, 'avg_raw_per_image': 31.05857142857143, 'avg_kept_per_image': 26.614285714285714, 'avg_removed_per_image': 4.444285714285714, 'nms_type': 'diou', 'conf': 0.385, 'diou_thr': 0.48, 'nms_iou': 0.5, 'imgsz': 832, 'device': 'cuda:0'}
              metric         value
0                MAE      1.664286
1               RMSE      4.379987
2  MAPE(ignore gt=0)      0.070440
3           N_images    700.000000
4           GT_total  18605.000000
5         Pred_total  18630.000000

=== DONE: outputs_grid_c385_diou49_20260113_033217 ===
predictions_txt: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\outputs_grid_c385_diou49_20260113_033217\predictions_txt (txt=700)
NMS stats: {'n_images': 700, 'raw_total': 21741, 'kept_to

KeyboardInterrupt: 

## Bu işlemde biz alacağımızı aldık. 0.39 - 0.51 doğru cevap

In [ ]:
from datetime import datetime

stamp = datetime.now().strftime("%Y%m%d_%H%M%S")

conf_list = [0.385, 0.390, 0.395]  # 0.39 ± 0.005
thr_list  = [0.48, 0.49, 0.50, 0.51, 0.52]

for conf in conf_list:
    for thr in thr_list:
        tag = f"micro_c{int(conf*1000)}_diou{int(thr*100)}_{stamp}"
        run_conf_nms(conf, tag, nms_type="diou", diou_thr=thr, nms_iou=0.50)


=== DONE: outputs_micro_c385_diou48_20260113_034114 ===
predictions_txt: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\outputs_micro_c385_diou48_20260113_034114\predictions_txt (txt=652)
NMS stats: {'n_images': 700, 'raw_total': 21741, 'kept_total': 18630, 'removed_total': 3111, 'avg_raw_per_image': 31.05857142857143, 'avg_kept_per_image': 26.614285714285714, 'avg_removed_per_image': 4.444285714285714, 'nms_type': 'diou', 'conf': 0.385, 'diou_thr': 0.48, 'nms_iou': 0.5, 'imgsz': 832, 'device': 'cuda:0'}
              metric         value
0                MAE      3.642857
1               RMSE     12.186527
2  MAPE(ignore gt=0)      0.135142
3           N_images    700.000000
4           GT_total  18605.000000
5         Pred_total  17223.000000

=== DONE: outputs_micro_c385_diou49_20260113_034114 ===
predictions_txt: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\outputs_micro_c385_diou49_20260113_034114\predictions_txt (txt=700)
NMS stats: {'n_images': 700, 'raw_total': 21741, 'kep

KeyboardInterrupt: 